### Import experimental data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

expt = np.genfromtxt('AC04RawData.csv', delimiter=',', names=['sx', 'sy']) #experimental data

### g(r) comparison

In [ ]:
with open('example.rdf') as f: #point to simulation folder
    n= 500 # number of points
    lines = (line for line in f if not line.startswith('#'))
    lines = f.readlines()
    lines = lines[3::]; del lines[0::n+1]; #skip the header
    data = np.loadtxt(lines)

total = np.zeros((n,3))
cut1 =int(len(data)/n)-1; cut2 = int(len(data)/n);
for i in np.arange(cut1,cut2):
    total += np.array_split(data, int(len(data)/n))[i]

averaged=total/(cut2-cut1)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))

plt.style.use('seaborn-notebook')

ax.plot(expt['sx'],expt['sy'],alpha=0.7,label='expt')
ax.plot(averaged[:,1],averaged[:,2],label='bd')
ax.set_xlim(3, 18)

x = np.arange(3.5, 6.5, 0.001)

### Visualisation & Cluster Analysis

In [ ]:
import MDAnalysis
print("Using MDAnalysis version", MDAnalysis.__version__)
d = MDAnalysis.coordinates.LAMMPS.DumpReader('brownian.lammpstrj')
pos = d._read_frame(-1).positions[:,:2]

In [ ]:
with open('example_data_file.txt') as f: #point to simulation folder
    lines = f.readlines()
    lines = lines[12::]; #skip the header
    data = np.loadtxt(lines)
    diams = data[:,5]

In [ ]:
# params
figdim = 7.5
box=200
figdim/box/2
ss = np.pi*(((diams*(figdim/box/2))*72)**2)*(4/np.pi)  # s params
np.mean(ss)

In [ ]:
fig,ax=plt.subplots(figsize=(7.5,7.5),frameon=False,dpi=72)
ax.set_xlim(0, box)
ax.set_ylim(0, box)
ax.scatter(pos[:,0],pos[:,1],s=ss,facecolors='#708090',edgecolors='none',linewidth=1)
ax.axis('off')
fig.tight_layout()
plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
plt.tight_layout(pad=0)
#plt.savefig('clustertest.png', format='png')

### Nearest neighbours

In [ ]:
salt=0.015
k1=(1/ (0.304/(np.sqrt(salt/1000))))*1000 #k1 in inverse microns, take from input file
1/k1 

In [ ]:
radii=diams/2
box=200 #box size in microns
nearest_neighbours = np.zeros((len(pos))) #initialise array to save to
for i in range(len(pos)): #pos is array of x,y coords shape [len(pos),2]
    
    pos2=np.abs(pos-pos[i])
    for j in range(len(pos2)):
        for k in range(2):
            if pos2[j,k]>box/2:
                pos2[j,k]=box-pos2[j,k] #account for pbc
    
    dists=np.linalg.norm(pos2,axis=1)
    idists = dists-radii-radii[i]
    nearest_neighbours[i]=len(idists[idists<(1/k1*10)])-1 #-1 as don't want to count self

In [ ]:
np.mean(nearest_neighbours)

In [ ]:
%config InlineBackend.figure_format = 'retina'
fig,ax=plt.subplots(figsize=(7.5,7.5),frameon=False,dpi=72)
ax = fig.add_axes([0, 0, 1, 1]) #this seems to be required see below
ax.set_xlim(0, 200)
ax.set_ylim(0, 200)
ax.scatter(pos[:,0],pos[:,1],s=ss,facecolors='none', edgecolors='b',linewidth=1, c=nearest_neighbours/6)
fig.tight_layout()
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)
plt.tight_layout(pad=0)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)